## 匯入套件 & 讀取資料

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import requests
import json
import csv
import numpy as np
import pandas as pd


activity_df = pd.read_csv('../data/preprocess/activity.csv')

# activity_df = total_df.loc[total_df.index <= 5]

activity_df

,id,title,subTitle,content,addTime,createAt,fee,holders,objectives,branches,tags,seg
0,badfa9cc-aed2-43df-4d56-08db52fbe02a,第12屆APX全國高中數理能力檢定,第12屆APX全國高中數理能力檢定,檢定分級級別檢定範圍及占分高級A部分，以高一到高三範圍為主，佔100分、B部分，以大學先修範...,NaN,2023-05-12T15:16:35.3314495,[],[],[],"[{'id': 131, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",檢定 分級 級別 檢定 範圍 占分 部分 高一 高三 範圍 為主 部分 大學 範圍 為主 中...
1,4625cc40-c153-457e-4d57-08db52fbe02a,青少年帆船2日體驗營,青少年帆船2日體驗營,陽光照耀下海面波光粼粼，徜徉在千變萬化的大海裡，體驗帆船的無限魅力，帶領孩子透過體驗及實務操...,NaN,2023-05-12T15:16:35.3350269,[],[],[],"[{'id': 132, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",陽光 照耀 海面 波光 粼粼 徜徉 千變萬化 大海 體驗 帆船 無限 魅力 帶領 孩子 體驗...
2,84841d32-fec3-4f35-4d58-08db52fbe02a,2023 成大觀影腦影像數據分析營 第三期,2023 成大觀影腦影像數據分析營 第三期,專業師資引導?分析演練實作招生對象全國高中生皆可以報名營隊時間2023/8/26(六)~20...,NaN,2023-05-12T15:16:35.3350899,[],[],[],"[{'id': 133, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",專業 師資 引導 分析 演練 實作 招生 對象 高中生 報名 營隊 時間 營隊 地點 成功 ...
3,81a759b6-6e15-465b-4d59-08db52fbe02a,2023 成大第十二期AI人才研修班,2023 成大第十二期AI人才研修班,課程期別：AI人才研修初階班(112/07/08週六–112/07/09週日)上課人數：上限...,NaN,2023-05-12T15:16:35.3360411,[],[],[],"[{'id': 134, 'branchName': 'General', 'locatio...",[],課程 期別 人才 研修 初階班 週六 週日 上課 人數 額滿 為止 適用 對象 高中 高職 ...
4,b5331379-6197-462b-4d5a-08db52fbe02a,臺大政治營,臺大政治營,【何謂模擬選戰】｜總統、國會選戰的精華版｜臺大政治營模擬選戰的設計理念是以「選舉」為主軸的大...,NaN,2023-05-12T15:16:35.3360799,[],[],[],"[{'id': 135, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",何謂 模擬 選戰 總統 國會 選戰 精華版 臺大 政治營 模擬 選戰 設計 理念 選舉 主軸...
...,...,...,...,...,...,...,...,...,...,...,...,...
355,73f16a9e-4aee-46f3-4ec3-08db52fbe02a,2022長庚大學職治營【職有你可以】,2022長庚大學職治營【職有你可以】,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!活動地點:長庚大學校園、長庚醫院桃園分院營隊時間...,NaN,2023-05-12T15:21:12.2716347,[],[],[],"[{'id': 497, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊 活動 地點 大學 校園 醫院 桃園 分院 營隊...
356,2b8d172d-fba4-43cb-4ec4-08db52fbe02a,2022東吳大學【全國高中生法律營】,2022東吳大學【全國高中生法律營】,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!活動時間：111/07/10~111/07/13...,NaN,2023-05-12T15:21:12.2716395,[],[],[],"[{'id': 498, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊 活動 時間 活動 地點 東吳 大學 外雙溪 校...
357,0abaa77e-3b32-4de4-4ec5-08db52fbe02a,2022輔仁大學心理營【between us】,2022輔仁大學心理營【between us】,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!,NaN,2023-05-12T15:21:12.2716426,[],[],[],"[{'id': 499, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊
358,a625a0e7-ffa9-4986-4ec6-08db52fbe02a,2022中山大學資工營,2022中山大學資工營,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!主辦單位：國立中山大學資訊工程學系活動對象：全國...,NaN,2023-05-12T15:21:12.2716453,[],[],[],"[{'id': 500, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊 主辦 單位 大學 資訊 工程 學系 活動 對象...


## 訓練 CKIP 模型

In [6]:
from ckip_transformers import __version__
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker


# Show version
print(__version__)

# Initialize drivers
print("Initializing drivers ... WS")
ws_driver = CkipWordSegmenter(model="albert-base")
print("Initializing drivers ... POS")
pos_driver = CkipPosTagger(model="albert-base")
print("Initializing drivers ... NER")
ner_driver = CkipNerChunker(model="albert-base")
print("Initializing drivers ... all done")
print()

C:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.3.4
Initializing drivers ... WS
Initializing drivers ... POS
Initializing drivers ... NER
Initializing drivers ... all done



## 清除沒必要的單詞

In [7]:
# clean function
def clean(sentence_ws, sentence_pos):
  short_with_pos = []
  short_sentence = []
  stop_pos = set(['Nep', 'Nh', 'Nb', 'Neu']) # 這 3 種詞性不保留 指代定詞 代名詞 專有名詞 數字
  for word_ws, word_pos in zip(sentence_ws, sentence_pos):
    # 只留名詞和動詞
    is_N_or_V = word_pos.startswith("V") or word_pos.startswith("N")
    # 去掉名詞裡的某些詞性
    is_not_stop_pos = word_pos not in stop_pos
    # 只剩一個字的詞也不留
    is_not_one_charactor = not (len(word_ws) == 1)
    
    # 組成串列
    if is_N_or_V and is_not_stop_pos and is_not_one_charactor:
      short_with_pos.append(f"{word_ws}({word_pos})")
      short_sentence.append(f"{word_ws}")
  return (" ".join(short_sentence), " ".join(short_with_pos))

## 執行斷詞

In [8]:
def CKIP_segment(content):
    ws = ws_driver([content])
    pos = pos_driver(ws)
    
    seg = ""
    for sentence_ws, sentence_pos in zip(ws, pos):
        (short, res) = clean(sentence_ws, sentence_pos)
        seg += short
    
    return seg

activity_df['seg'] = activity_df['content'].apply(CKIP_segment)

activity_df

print("done!")

Tokenization: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 972.25it/s]


Inference: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.05it/s]


Tokenization: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 692.82it/s]


Inference: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.97s/it]


Tokenization: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1000.31it/s]


...

Tokenization: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Inference: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.77s/it]


Tokenization: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Inference: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.16s/it]


,id,title,subTitle,content,addTime,createAt,fee,holders,objectives,branches,tags,seg
0,badfa9cc-aed2-43df-4d56-08db52fbe02a,第12屆APX全國高中數理能力檢定,第12屆APX全國高中數理能力檢定,檢定分級級別檢定範圍及占分高級A部分，以高一到高三範圍為主，佔100分、B部分，以大學先修範...,NaN,2023-05-12T15:16:35.3314495,[],[],[],"[{'id': 131, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",檢定 分級 級別 檢定 範圍 占分 部分 高一 高三 範圍 為主 部分 大學 範圍 為主 中...
1,4625cc40-c153-457e-4d57-08db52fbe02a,青少年帆船2日體驗營,青少年帆船2日體驗營,陽光照耀下海面波光粼粼，徜徉在千變萬化的大海裡，體驗帆船的無限魅力，帶領孩子透過體驗及實務操...,NaN,2023-05-12T15:16:35.3350269,[],[],[],"[{'id': 132, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",陽光 照耀 海面 波光 粼粼 徜徉 千變萬化 大海 體驗 帆船 無限 魅力 帶領 孩子 體驗...
2,84841d32-fec3-4f35-4d58-08db52fbe02a,2023 成大觀影腦影像數據分析營 第三期,2023 成大觀影腦影像數據分析營 第三期,專業師資引導?分析演練實作招生對象全國高中生皆可以報名營隊時間2023/8/26(六)~20...,NaN,2023-05-12T15:16:35.3350899,[],[],[],"[{'id': 133, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",專業 師資 引導 分析 演練 實作 招生 對象 高中生 報名 營隊 時間 營隊 地點 成功 ...
3,81a759b6-6e15-465b-4d59-08db52fbe02a,2023 成大第十二期AI人才研修班,2023 成大第十二期AI人才研修班,課程期別：AI人才研修初階班(112/07/08週六–112/07/09週日)上課人數：上限...,NaN,2023-05-12T15:16:35.3360411,[],[],[],"[{'id': 134, 'branchName': 'General', 'locatio...",[],課程 期別 人才 研修 初階班 週六 週日 上課 人數 額滿 為止 適用 對象 高中 高職 ...
4,b5331379-6197-462b-4d5a-08db52fbe02a,臺大政治營,臺大政治營,【何謂模擬選戰】｜總統、國會選戰的精華版｜臺大政治營模擬選戰的設計理念是以「選舉」為主軸的大...,NaN,2023-05-12T15:16:35.3360799,[],[],[],"[{'id': 135, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",何謂 模擬 選戰 總統 國會 選戰 精華版 臺大 政治營 模擬 選戰 設計 理念 選舉 主軸...
...,...,...,...,...,...,...,...,...,...,...,...,...
355,73f16a9e-4aee-46f3-4ec3-08db52fbe02a,2022長庚大學職治營【職有你可以】,2022長庚大學職治營【職有你可以】,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!活動地點:長庚大學校園、長庚醫院桃園分院營隊時間...,NaN,2023-05-12T15:21:12.2716347,[],[],[],"[{'id': 497, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊 活動 地點 大學 校園 醫院 桃園 分院 營隊...
356,2b8d172d-fba4-43cb-4ec4-08db52fbe02a,2022東吳大學【全國高中生法律營】,2022東吳大學【全國高中生法律營】,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!活動時間：111/07/10~111/07/13...,NaN,2023-05-12T15:21:12.2716395,[],[],[],"[{'id': 498, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊 活動 時間 活動 地點 東吳 大學 外雙溪 校...
357,0abaa77e-3b32-4de4-4ec5-08db52fbe02a,2022輔仁大學心理營【between us】,2022輔仁大學心理營【between us】,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!,NaN,2023-05-12T15:21:12.2716426,[],[],[],"[{'id': 499, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊
358,a625a0e7-ffa9-4986-4ec6-08db52fbe02a,2022中山大學資工營,2022中山大學資工營,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!主辦單位：國立中山大學資訊工程學系活動對象：全國...,NaN,2023-05-12T15:21:12.2716453,[],[],[],"[{'id': 500, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊 主辦 單位 大學 資訊 工程 學系 活動 對象...


## 存入資料

In [16]:
activity_df.to_csv('../data/preprocess/activity.csv', index=False)

activity_df = pd.read_csv('../data/preprocess/activity.csv')

activity_df

,id,title,subTitle,content,addTime,createAt,fee,holders,objectives,branches,tags,seg
0,badfa9cc-aed2-43df-4d56-08db52fbe02a,第12屆APX全國高中數理能力檢定,第12屆APX全國高中數理能力檢定,檢定分級級別檢定範圍及占分高級A部分，以高一到高三範圍為主，佔100分、B部分，以大學先修範...,NaN,2023-05-12T15:16:35.3314495,[],[],[],"[{'id': 131, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",檢定 分級 級別 檢定 範圍 占分 部分 高一 高三 範圍 為主 部分 大學 範圍 為主 中...
1,4625cc40-c153-457e-4d57-08db52fbe02a,青少年帆船2日體驗營,青少年帆船2日體驗營,陽光照耀下海面波光粼粼，徜徉在千變萬化的大海裡，體驗帆船的無限魅力，帶領孩子透過體驗及實務操...,NaN,2023-05-12T15:16:35.3350269,[],[],[],"[{'id': 132, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",陽光 照耀 海面 波光 粼粼 徜徉 千變萬化 大海 體驗 帆船 無限 魅力 帶領 孩子 體驗...
2,84841d32-fec3-4f35-4d58-08db52fbe02a,2023 成大觀影腦影像數據分析營 第三期,2023 成大觀影腦影像數據分析營 第三期,專業師資引導?分析演練實作招生對象全國高中生皆可以報名營隊時間2023/8/26(六)~20...,NaN,2023-05-12T15:16:35.3350899,[],[],[],"[{'id': 133, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",專業 師資 引導 分析 演練 實作 招生 對象 高中生 報名 營隊 時間 營隊 地點 成功 ...
3,81a759b6-6e15-465b-4d59-08db52fbe02a,2023 成大第十二期AI人才研修班,2023 成大第十二期AI人才研修班,課程期別：AI人才研修初階班(112/07/08週六–112/07/09週日)上課人數：上限...,NaN,2023-05-12T15:16:35.3360411,[],[],[],"[{'id': 134, 'branchName': 'General', 'locatio...",[],課程 期別 人才 研修 初階班 週六 週日 上課 人數 額滿 為止 適用 對象 高中 高職 ...
4,b5331379-6197-462b-4d5a-08db52fbe02a,臺大政治營,臺大政治營,【何謂模擬選戰】｜總統、國會選戰的精華版｜臺大政治營模擬選戰的設計理念是以「選舉」為主軸的大...,NaN,2023-05-12T15:16:35.3360799,[],[],[],"[{'id': 135, 'branchName': 'General', 'locatio...","[{'trend': 0, 'tagVoteCount': 0, 'userVoted': ...",何謂 模擬 選戰 總統 國會 選戰 精華版 臺大 政治營 模擬 選戰 設計 理念 選舉 主軸...
...,...,...,...,...,...,...,...,...,...,...,...,...
355,73f16a9e-4aee-46f3-4ec3-08db52fbe02a,2022長庚大學職治營【職有你可以】,2022長庚大學職治營【職有你可以】,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!活動地點:長庚大學校園、長庚醫院桃園分院營隊時間...,NaN,2023-05-12T15:21:12.2716347,[],[],[],"[{'id': 497, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊 活動 地點 大學 校園 醫院 桃園 分院 營隊...
356,2b8d172d-fba4-43cb-4ec4-08db52fbe02a,2022東吳大學【全國高中生法律營】,2022東吳大學【全國高中生法律營】,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!活動時間：111/07/10~111/07/13...,NaN,2023-05-12T15:21:12.2716395,[],[],[],"[{'id': 498, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊 活動 時間 活動 地點 東吳 大學 外雙溪 校...
357,0abaa77e-3b32-4de4-4ec5-08db52fbe02a,2022輔仁大學心理營【between us】,2022輔仁大學心理營【between us】,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!,NaN,2023-05-12T15:21:12.2716426,[],[],[],"[{'id': 499, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊
358,a625a0e7-ffa9-4986-4ec6-08db52fbe02a,2022中山大學資工營,2022中山大學資工營,相關問題請至此營隊粉絲團，查詢聯絡人資訊!!主辦單位：國立中山大學資訊工程學系活動對象：全國...,NaN,2023-05-12T15:21:12.2716453,[],[],[],"[{'id': 500, 'branchName': 'General', 'locatio...",[],相關 問題 營隊 粉絲團 查詢 聯絡人 資訊 主辦 單位 大學 資訊 工程 學系 活動 對象...


## 設立使用者資料

In [9]:
# 假設使用者活動紀錄
userli = [[1, 2], [1, 2, 4], [3]]

user_seg = [' '.join(activity_df.loc[userli[1], 'seg'].values.tolist())]

user_seg

['陽光 照耀 海面 波光 粼粼 徜徉 千變萬化 大海 體驗 帆船 無限 魅力 帶領 孩子 體驗 實務 操作 學習 帆船 相關 知識 培養 隊員 探索 解決 問題 團隊 合作 無限 潛能 駕馭 風帆 享受 勇敢 航向 大海 學習 目標 認識 帆船 運動 操作 認識 帆船 部件 知道 方位 知道 帆船 術語 實作 帆船 結繩 掌舵 讓船 航行 團隊 分工 共作 學習 目標 操作 重型 帆船 發展 團隊 合作 關係 操作 帆船 部件 理解 方位 熟悉 帆船 術語 實作 帆船 結繩 操作 帆讓船 航行 團隊 分工合作 活動 場域 西子灣 哨船頭 遊艇 碼頭 安平港 安平 漁港 遊艇 碼頭 興達港 興達港 情人 碼頭 遊艇 碼頭 專業 師資 引導 分析 演練 實作 招生 對象 高中生 報名 營隊 時間 營隊 地點 成功 大學 社會 科學院 三樓 演講廳 三樓 電腦 教室 授課 講師 成功 大學 心理學系 教授 成功 大學 心智 影像 研究 中心 大學 認知 科學 博士 普林斯頓 大學 神經 科學 研究院 博士 研究 研究 興趣 臉孔 物體 辨識 決策 神經 科學 開放 神經 科學 開設 相關 課程 認知 神經 科學 電影 功能性 磁振 造影 理論 實務 功能性 磁振 造影 進階 分析 何謂 模擬 選戰 總統 國會 選戰 精華版 臺大 政治營 模擬 選戰 設計 理念 選舉 主軸 遊戲 營期 真實 總統 國會 公投案 選舉 精要 濃縮版 政策 公投案 擬定 遊說 政治 學長姐 扮演 NPC 政見 發表 造勢 跨隊 結盟 手段 模擬 政壇 生態 目標 獲得 選舉 勝利 選擇 角色 大政 奉還 小隊 配置 分為 政黨 小隊 利益 團體 屬於 捍衛 信念 推動 公投案 利團 取得 執政權 改變 政治 現狀 政黨 這裡 發揮 影響力 憑藉 謀略 膽識 獲得 一席之地 豐富 營隊 體驗 激烈 模擬 選戰 參與 臺灣 政治 文化 常見 綁樁 結盟 政論 節目 造勢 晚會 精彩 活動 政黨 利團 發布 媒體 新聞 宣傳 理念 官方 媒體 播報 即時 新聞 擔任 公正 角色 地方 勢力 富商 權貴 勞動 階級 小隊員 倚靠 謹惕 對象 選舉 走向 具有 影響力 課程 安排 學術 顧問 小隊 主題 專題 演講 邀集 臺灣 政壇 重量級 講者 學員 進行 主題式 演講 隊員 課程 議題 事務 獲得

## 算出使用者和活動的 TF-IDF 特徵矩陣，並算出相似值

In [20]:
vectorizer = TfidfVectorizer()

# 目的是學習每個單詞在所有文本中的重要性，轉換的目的是將每個文本轉換成一個數值向量
activity_vec = vectorizer.fit_transform(activity_df['seg'].values)

# 已經有一個訓練好的向量化模型，並且想要將新的文本資料轉換成與這個模型相同的向量形式
user_vec = vectorizer.transform(user_seg)

# 計算使用者向量和語料庫中所有活動向量之間的餘弦相似度
user_activity_matrix = cosine_similarity(user_vec, activity_vec)

# 透過增加字元的方式調整單詞權重

user_activity_matrix = pd.DataFrame(user_activity_matrix, index=[1], columns=activity_df.index)

user_activity_matrix

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
1,0.002619,0.630266,0.36123,0.05665,0.700741,0.060074,0.089663,0.037791,0.061611,0.012655,...,0.022603,0.022543,0.017937,0.017885,0.00898,0.025576,0.021879,0.024496,0.033204,0.07454


In [33]:
activity_content = activity_df['seg'].values

### 特徵權重

In [22]:
# 獲取詞彙列表
vocab = vectorizer.get_feature_names_out()

# 獲取詞彙權重
weights = activity_vec.toarray()[0]  # 假設只有一個文本，取第一個向量
word_weights = list(zip(vocab, weights))

# 輸出詞彙權重
for word, weight in word_weights:
    if weight > 0:
        print(word, weight)

下載 0.11605814466729687
中級 0.09498634510527848
中高級 0.14247951765791772
以上 0.2931671121866189
分數 0.3799453804211139
分級 0.04749317255263924
初級 0.09498634510527848
化學科 0.14850163085236937
占分 0.049500543617456455
合格 0.6435070670269339
大學 0.013616149441908995
應試 0.049500543617456455
採計 0.049500543617456455
數學科 0.14850163085236937
方式 0.016315852264399695
標準 0.16161270807426362
檢定 0.07779502574558146
為主 0.1880509792333636
物理科 0.14850163085236937
生物科 0.14850163085236937
範圍 0.22566117508003636
級別 0.049500543617456455
計算 0.03823084696155322
證書 0.12099512016606588
達到 0.036484244099164384
部分 0.18514807751080317
高一 0.1350598549728612
高三 0.06676420570768406
高二 0.04220551832081924
高級 0.09170606193827568


特徵選擇

In [26]:
# 使用 TfidfVectorizer 創建詞彙向量，限制最大特徵數量
vectorizer = TfidfVectorizer(max_features=100)  # 假設只保留100個最重要的詞彙
activity_vec = vectorizer.fit_transform(activity_df['seg'].values)

# 獲取詞彙列表
vocab = vectorizer.get_feature_names_out()

# 輸出特徵選擇後的詞彙列表
print(vocab)

['上課' '不同' '主辦' '了解' '介紹' '以上' '任何' '住宿' '使用' '優惠' '內容' '分享' '包含' '匯款'
 '即日' '參加' '參與' '台灣' '合作' '名額' '問題' '單位' '團報' '團隊' '地點' '報名' '報名表' '填寫'
 '大學' '孩子' '學員' '學生' '學習' '完成' '家長' '實作' '專業' '對象' '思考' '應用' '招生' '探索'
 '提供' '教學' '教授' '教育' '方式' '方案' '日期' '早鳥' '時間' '智慧' '暑假' '期間' '未來' '梯次'
 '歡迎' '歷程' '活動' '營隊' '環境' '生活' '發展' '目標' '相關' '知識' '科學' '科技' '管理' '簡章'
 '經驗' '網路' '線上' '繳費' '聯絡' '能力' '興趣' '表單' '表達' '訓練' '設計' '認識' '課程' '證明'
 '豐富' '費用' '資料' '資訊' '退費' '通知' '連結' '進行' '遊戲' '醫學' '錄取' '開始' '領域' '體驗'
 '高中' '高中生']


In [35]:
from sklearn.feature_selection import SelectKBest, chi2

# 使用 TfidfVectorizer 創建詞彙向量
vectorizer = TfidfVectorizer()
activity_vec = vectorizer.fit_transform(activity_content)

# 使用卡方檢定進行特徵選擇
k_best = SelectKBest(chi2, k=100)  # 假設保留100個最重要的詞彙
selected_features = k_best.fit_transform(activity_vec, labels)

# 獲取選擇後的詞彙列表
vocab = vectorizer.get_feature_names()
selected_vocab = [vocab[i] for i in k_best.get_support(indices=True)]

# 輸出選擇後的詞彙列表
print(selected_vocab)


NameError: name 'labels' is not defined

## 找出最相似的活動

In [227]:
def get_the_most_similar_activitys(user_id, user_movie_matrix, num):
    """Find the top-n movies most similar to the user"""
    user_vec = user_movie_matrix.loc[user_id].values 
    sorted_index = np.argsort(user_vec)[::-1][:num]
    return list(user_movie_matrix.columns[sorted_index])

activity_ids = get_the_most_similar_activitys(0, user_activity_matrix, 3)

# print(activity_df[activity_df.index.isin(activity_ids)]["title"])
# print(activity_df[activity_df.index.isin(activity_ids)]['activity_id'])


# activity_df[activity_df.index.isin(activity_ids)]
activity_ids


[4, 1, 2]

In [149]:
import pandas as pd
import numpy as np

# 創建一個 5x3 的隨機數據矩陣
data = np.random.randn(5, 3)

# 將隨機數據矩陣轉換為 DataFrame
df = pd.DataFrame(data, columns=['A', 'B', 'C'])

# 輸出 DataFrame
df.loc[[1, 2, 4], 'B']

1   -0.611840
2    1.356513
4   -0.630494
Name: B, dtype: float64